# Matematikken bag Neurale Netværk
## Bagtanker

Her er der opgaver som er lavet til at give jer en fuldstændig forståelse for hvordan Neurale Netværk virker. Det sker ved at I skal bygge et og optimere med numpy som ikke er bygget til det så i skal selv lave beregningerne med matricerne.

## Pakker
Her er numpy og matplot som er alle de pakker i har brug for, for at løse opgaverne her.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Vægte

Her skal i lave to funktioner, en som initialisere tilfældige vægte og en som laver et netværk udfra de vægte.

**Hvad er vægte og bias:**
- **Vægte (W):** Bestemmer hvor stærk forbindelsen er mellem neuroner i forskellige lag
- **Bias (b):** Tillader netværket at forskyde aktiveringsgrænsen for hver neuron
- Vægte er matricer med dimensioner (input_size, output_size)
- Bias er vektorer med længde output_size

**Initialisering:**
- Vægte initialiseres med tilfældige værdier (brug `np.random.randn()`)
- Bias kan initialiseres til nul (brug `np.zeros()`)
- Korrekte dimensioner er kritiske for matrix multiplikation

**Tips:**
- For `init_weight`: returner en vægt-matrix (dim_in × dim_out) og bias-vektor (dim_out)
- For `init_NN`: brug et loop til at oprette vægte for hvert lag
- Husk at det første lag tager X_dim som input, derefter bruger du L værdierne

In [ ]:
def init_weight(dim_in: int, dim_out: int):
    # TODO: Her skal du initialisere vægtene som matrice og bias som vektor for et lag i et neuralt netværk.
    W = np.random.randn(dim_out, dim_in)
    b = np.zeros(dim_out)
    return W, b

def init_NN(X_dim: int, L: list[int]):
    weights, biases = [], []
    prev_dim = X_dim

    for out_dim in L:
        W, b = init_weight(prev_dim, out_dim)
        weights.append(W)
        biases.append(b)
        prev_dim = out_dim

    return weights, biases

In [ ]:
def print_NN_params(weights: list, biases: list):
    print(f"Netværket har {len(weights)} lag, med hhv. {', '.join([f'{weights[i].shape[1]} neuroner i lag ({i+1})' for i in range(len(weights))])}")
    print()
    for i, (weight, bias) in enumerate(zip(weights, biases)):
        print(f'W^({i+1}):')
        print(weight)
        print(f'b^({i+1}):')
        print(bias)
        print()

print_NN_params(*init_NN(X_dim=3, L = [2, 1]))

## Aktiveringsfunktioner

Her skal I implementere de forskellige aktiveringsfunktioner. Aktiveringsfunktioner bestemmer om en neuron skal "aktiveres" eller ej, og introducerer non-linearitet i netværket.

**Hvad gør aktiveringsfunktioner:**
- Tager input z (linear transformation) og transformerer det til en aktivering a
- Introducerer non-linearitet - uden dem ville netværket kun kunne lære lineære sammenhænge
- Forskellige funktioner har forskellige egenskaber og anvendelser

**De forskellige funktioner:**
- **ReLU:** Simpel og hurtig, men kan "dø" (gradient bliver 0)
- **Leaky ReLU:** Som ReLU men undgår "døde" neuroner
- **Sigmoid:** Squasher output til [0,1], men kan have vanishing gradient problem
- **Tanh:** Som sigmoid men output er [-1,1]
- **Softmax:** Bruges til klassifikation - konverterer til sandsynligheder

Vi har lavet de afledte for jer som i kan tage inspiration fra.

In [ ]:
def ReLu(z: np.ndarray, return_derivative: bool = False) -> np.ndarray:
    if return_derivative:
        return np.where(z > 0, 1, 0)
    else:
        # TODO: Implementer ReLu
        return ...

def tanh(z: np.ndarray, return_derivative: bool = False) -> np.ndarray:
    if return_derivative:
        return 1 - tanh(z)**2
    else:
        # TODO: Implementer tanh
        return ...

def sigmoid(z: np.ndarray, return_derivative: bool = False) -> np.ndarray:
    if return_derivative:
        return sigmoid(z) * (1 - sigmoid(z))
    else:
        # TODO: Implementer sigmoid
        return ...

def leaky_ReLu(z: np.ndarray, alpha: float = 0.1, return_derivative: bool = False) -> np.ndarray:
    if return_derivative:
        return np.where(z > 0, 1, alpha)
    else:
        # TODO: Implementer leaky ReLu
        return ...

def softmax(z: np.ndarray, return_derivative: bool = False) -> np.ndarray:
    if return_derivative:
        return softmax(z) * (np.ones(z.shape) - softmax(z))
    else:
        # TODO: Implementer softmax
        return ...

In [ ]:
activation_funcs = [tanh, sigmoid, ReLu, leaky_ReLu]
fig, ax = plt.subplots(1, len(activation_funcs), figsize=(15, 4), sharey=True, layout='tight')
ax[0].set_ylabel('$f(z)$')
z = np.linspace(-5, 5, 100).reshape(-1, 1)
for i, f in enumerate(activation_funcs):
    ax[i].plot(z, f(z), label=f.__name__)
    ax[i].plot(z, f(z, return_derivative=True), '--', color='tab:blue')
    ax[i].set_title(f.__name__)
    ax[i].plot(z, np.zeros_like(z), 'k--', linewidth=0.5)
    ax[i].plot(np.zeros_like(z), z, 'k--', linewidth=0.5)
    ax[i].set_ylim([-2, 2])
    ax[i].set_xlim([-5, 5])
    ax[i].grid(True)
    ax[i].set_xlabel('$z$')
plt.show()

## Fremadpropagering (Forward Propagation)

Her skal I færdiggøre fremadpropagering, som er processen hvor data flyder gennem netværket fra input til output.

**Hvad sker der i hvert lag:**
1. **Linear transformation:** $$z = X \cdot W + b$$
2. **Aktivering:** $$a = f(z)$$

**Proces:**
- Tag input data (X) 
- Gang med vægte (W) og læg bias (b) til → dette giver z
- Anvend aktiveringsfunktion på z → dette giver a (aktivering)
- a bliver input til næste lag

**Tips:**
- Brug `np.dot` for matrix multiplikation
- Husk at tilføje bias: `+ b`
- Anvend aktiveringsfunktionen
- Den sidste aktivering er dit output

In [ ]:
def forward(X: np.ndarray, weights: list, biases: list, activation_funcs: list) -> np.ndarray:
    a = X.T
    for i, (W, b, f) in enumerate(zip(weights, biases, activation_funcs)):
        # TODO: Implementer fremadpropagering i det neurale netværk.
        ...
    return a

In [ ]:
X = np.array([[0.2, 0.8], [2.5, 3], [0.3, -0.9],[4, 3.5]])
weights, biases = init_NN(X_dim=2, L=[3, 3, 3, 3, 2])
print(biases[0].shape)
fs = [tanh, ReLu, leaky_ReLu, ReLu, softmax]
print(forward(X, weights, biases, fs))

In [ ]:
def get_label(prob: np.ndarray) -> np.ndarray:
    return np.argmax(prob, axis=1)

In [ ]:
y_prob = forward(X, weights, biases, fs)
y = get_label(y_prob)
print(f"Klassifikation: {y}")
plt.scatter(X[:, 0], X[:, 1])
plt.show()

## Loss funktioner
Her skal i implementere nogle forskellige loss funktioner

### Mean Squared Error (MSE)
**Matematik:** $$MSE = \frac{1}{n} \sum_{i=1}^{n}(y_{true} - y_{pred})^2$$
- Bruges til regression problemer
- Straffer store fejl mere end små fejl (kvadratisk)

### Mean Absolute Error (MAE) 
**Matematik:** $$MAE = \frac{1}{n} \sum_{i=1}^{n}|y_{true} - y_{pred}|$$
- Bruges til regression problemer
- Mere robust overfor outliers end MSE

### Categorical Cross-Entropy (CCE)
**Matematik:** $$CCE = -\frac{1}{n} \sum_{i=1}^{n} \sum_{j=1}^{C} y_{true,i,j} \log(y_{pred,i,j})$$
- Bruges til multi-class klassifikation
- y_true skal være one-hot encoded
- y_pred skal være sandsynligheder (softmax output)

### Binary Cross-Entropy (BCE)
**Matematik:** $$BCE = -\frac{1}{n} \sum_{i=1}^{n}[y_{true,i} \log(y_{pred,i}) + (1-y_{true,i}) \log(1-y_{pred,i})]$$
- Bruges til binær klassifikation  
- y_true skal være 0 eller 1
- y_pred skal være sandsynlighed mellem 0 og 1 (sigmoid output)

**Tips til implementering:**
- Brug `np.mean()` for gennemsnit
- Tilføj en lille epsilon (f.eks. 1e-15) til log for at undgå log(0)

In [ ]:
def MSE(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    # TODO:
    return ...

def MAE(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    # TODO:
    return 1/len(y_true) * np.sum(abs(y_true - y_pred))

def CCE(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    # TODO:
    y_pred_clipped = np.clip(y_pred, 1e-15, 1.0)
    return -np.mean(np.sum(y_true * np.log(y_pred_clipped), axis=1))

def BCE(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    # TODO:
    y_pred_clipped = np.clip(y_pred, 1e-15, 1.0 - 1e-15)
    return -np.mean(
        y_true * np.log(y_pred_clipped) +
        (1 - y_true) * np.log(1 - y_pred_clipped)
    )

## Data

**Ingen opgave her - bare kør koden!**

Her genererer vi noget syntetisk data som I kan klassificere. Funktionen `data_generator()` laver 3 klasser af datapunkter, hvor hver klasse er centreret omkring forskellige punkter med lidt støj tilføjet. 

**Hvad sker der:**
- Klasse 0: Data centreret omkring (0, 0)
- Klasse 1: Data centreret omkring (1, 1) 
- Klasse 2: Data centreret omkring (2, 2)
- Støj tilføjes for at gøre klassifikationen mere udfordrende
- Data opdeles i træning (60%), validering (20%) og test (20%)

Dette giver jer et simpelt klassifikationsproblem at teste jeres neurale netværk på.

In [ ]:
def data_generator(n_datapunkter: int = 3000, n_klasser: int = 3, n_dim: int = 2, støj: float = 0.9):
    for klasse in range(n_klasser):
        X_ = np.random.normal(klasse, støj, (n_datapunkter // n_klasser, n_dim))
        y_ = np.full(X_.shape[0], klasse)
        if klasse == 0:
            X = X_
            y = y_
        else:
            X = np.vstack([X, X_])
            y = np.hstack([y, y_])

    idx = np.random.permutation(X.shape[0])
    X, y = X[idx], y[idx]
    X_train, y_train = X[:int(0.6 * X.shape[0])], y[:int(0.6 * X.shape[0])]
    X_val, y_val = X[int(0.6 * X.shape[0]):int(0.8 * X.shape[0])], y[int(0.6 * X.shape[0]):int(0.8 * X.shape[0])]
    X_test, y_test = X[int(0.8 * X.shape[0]):], y[int(0.8 * X.shape[0]):]
    return X_train, y_train, X_val, y_val, X_test, y_test  

def plot_data(X: np.ndarray, y: np.ndarray, axs: plt.Axes = None):
    if axs is None:
        fig, axs = plt.subplots(1, 1, figsize=(5, 5))
    for klasse in np.unique(y):
        axs.scatter(X[y == klasse, 0], X[y == klasse, 1], label=f'Klasse {klasse}')
    axs.legend()

X_train, y_train, X_val, y_val, X_test, y_test = data_generator()
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
for i, x, y in zip(range(3), [X_train, X_val, X_test], [y_train, y_val, y_test]):
    plot_data(x, y, ax[i])
    ax[i].set_title(['Træningsdata', 'Valideringsdata', 'Testdata'][i])
    ax[i].set_xlabel('$x_1$')
    ax[i].set_ylabel('$x_2$')
plt.show()

# Træning

Her skal i færdiggøre træningen af jeres neurale netværk. Træning består af tre hovedkomponenter:

## Backpropagation (Baglæns propagering)
**Formål:** Beregne hvor meget hver vægt og bias skal ændres for at minimere fejlen.

**Matematik:** Backpropagation bruger kædereglen til at beregne gradienter:
- **Output lag:** $$\delta^{(L)} = \frac{dL}{dz^{(L)}} = a^{(L)} - y_{true}$$ (for softmax + cross-entropy)
- **Skjulte lag:** $$\delta^{(l)} = (W^{(l+1)})^T \delta^{(l+1)} \times f'(z^{(l)})$$ (element-wise multiplikation)

**Gradienter:**
- **Vægte:** $$\frac{dL}{dW^{(l)}} = a^{(l-1)} (\delta^{(l)})^T$$
- **Bias:** $$\frac{dL}{db^{(l)}} = \delta^{(l)}$$

## Parameterudatering
**Gradient Descent:** Opdater parametre i den modsatte retning af gradienten:
- $$W^{(l)} = W^{(l)} - \alpha \frac{dL}{dW^{(l)}}$$
- $$b^{(l)} = b^{(l)} - \alpha \frac{dL}{db^{(l)}}$$

hvor $\alpha$ er learning rate (læringsraten).

## Træningsloop
1. **Forward pass:** Beregn output og aktivationer
2. **Beregn loss:** Sammenlign output med rigtige labels
3. **Backward pass:** Beregn gradienter
4. **Opdater parametre:** Anvend gradient descent
5. **Gentag** for hver epoch

**Tips til implementering:**
- Husk at matricerne skal have de rigtige dimensioner for matrix multiplikation
- Brug `np.dot()` for matrix multiplikation mellem aktivationer og vægte
- Forward funktionen skal returnere både aktivationer og z-værdier for backpropagation
- For accuracy: sammenlign `np.argmax(y_pred, axis=1)` med rigtige labels
- Print shapes af matricer for at debugge dimensionsfejl
- Start med simple netværk (få lag) for at teste implementeringen

In [ ]:
X_dim = X_train.shape[1]
L = [10, 10, 10, 3] # TODO: Definer lagstørrelserne for det neurale netværk, f.eks. [10, 10, 10, 3] for et netværk med 3 skjulte lag og 3 output neuroner.
weights, biases = init_NN(X_dim, L)
fs = [ReLu, leaky_ReLu, ReLu, softmax] # TODO: Definer aktiveringsfunktionerne for hvert lag i det neurale netværk, f.eks. [ReLu, leaky_ReLu, ReLu, softmax] for et netværk med 3 skjulte lag og 1 output neuron.

In [ ]:
# Sæt weights her
def backward(y_true, activations, zs, weights, biases, activation_funcs):
    deltas = [None] * len(weights)
    grads_w = [None] * len(weights)
    grads_b = [None] * len(biases)

    # Compute the delta for the last layer
    delta = # TODO: Beregn delta for det sidste lag, f.eks. ved at bruge softmax og CCE.
    deltas[-1] = delta

    # Backpropagate the error
    for l in range(len(weights)-2, -1, -1):
        delta = # TODO: beregn delta for det aktuelle lag.
        deltas[l] = delta

    # Compute gradients
    for l in range(len(weights)):
        grads_w[l] = # TODO: beregn gradienten for vægtene i det aktuelle lag.
        grads_b[l] = # TODO: beregn gradienten for bias i det aktuelle lag.

    return grads_w, grads_b

def update_parameters(weights, biases, grads_w, grads_b, learning_rate):
    for l in range(len(weights)):
        # TODO: Opdater vægte og bias for hvert lag i det neurale netværk. Hvor de er vaegtet med læringsraten.
        pass
    return weights, biases

def train(X, y, weights, biases, activation_funcs, epochs, learning_rate):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    # One-hot encode y_true_train and y_true_val
    N_klasser = np.unique(y).shape[0]
    N_samples = y.shape[0]

    y_true_one_hot = np.zeros((N_samples, N_klasser))
    y_true_one_hot[np.arange(len(y)), y] = 1
    y_val_one_hot = np.zeros((y_val.shape[0], N_klasser))
    y_val_one_hot[np.arange(len(y_val)), y_val] = 1

    for epoch in range(epochs):
        activations, zs = # TODO: Implementer fremadpropagering i det neurale netværk for træningsdata.
        grads_w, grads_b = backward(y_true_one_hot, activations, zs, weights, biases, activation_funcs)

        # Evaluate the loss
        train_loss = # TODO implementer en loss funktion, f.eks. CCE
        train_losses.append(train_loss)
        # TODO: Tilføj en accuracy funktion for træningsdata og tilføj den til train_accuracies

        val_activations, _ = # TODO: Implementer fremadpropagering i det neurale netværk for valideringsdata.
        val_loss = # TODO implementer en loss funktion, f.eks. CCE
        val_losses.append(val_loss)
        # TODO: Tilføj en accuracy funktion for valideringsdata og tilføj den til val_accuracies

        weights, biases = update_parameters(weights, biases, grads_w, grads_b, learning_rate)
        if epoch % 5 == 0:
            print(f'Epoch {epoch}{" " if len(str(epoch)) == 1 else ""} ### Train Loss: {train_loss} ### Val Loss: {val_loss}')
    
    metrics = {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies
    }
    return weights, biases, metrics

In [ ]:
[print(weight.shape) for weight in weights]

# Choose hyperparameters
epochs = 30
learning_rate = 0.0001

weights, biases, metrics = train(X_train, y_train, weights, biases, fs, epochs=epochs, learning_rate=learning_rate)

fig, ax = plt.subplots(1, 2, figsize=(15, 5), layout='tight')
ax[0].plot(metrics['train_losses'], label='Training', linestyle='--', color='tab:blue')
ax[0].plot(metrics['val_losses'], label='Validation', color='tab:orange')
ax[0].set_title("Loss")

ax[1].plot(metrics['train_accuracies'], label='Training', linestyle='--', color='tab:blue')
ax[1].plot(metrics['val_accuracies'], label='Validation', color='tab:orange')
ax[1].set_title("Accuracy")

for a in ax:
    a.set_xlabel('Epoch')

# shared legend below the plots
lines, labels = ax[0].get_legend_handles_labels()
fig.legend(lines, labels, loc='lower center', ncol=2)
plt.show()